### Question 2

The image calibrated via Aladin gives us 5:35:21:650, -5:18:51.500.<br>
The calibration via Astrometry gives us 5:35:31.421, -5:16:00.961.<br><br>
The RA is different by arcsecond and the Dec is off by 2 arcminutes. The difference could be caused by human error in that the wrong stars were chosen for calibration, or maybe imaging error due to all the stars being stretched diagonally.

In [61]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from glob import glob

In [62]:
ls M42_Feb2017_Images\\

 Volume in drive C is Windows
 Volume Serial Number is D878-4535

 Directory of C:\Users\Rose\Documents\ASTR250\ICE16\M42_Feb2017_Images

04/13/2018  08:35 PM    <DIR>          .
04/13/2018  08:35 PM    <DIR>          ..
03/24/2018  11:42 PM        16,663,680 M42_147_R_calibrated.fits
03/20/2018  05:13 PM        16,663,680 M42_20180227_00000141_R.fits
03/20/2018  05:13 PM        16,663,680 M42_20180227_00000142_G.fits
03/20/2018  05:13 PM        16,663,680 M42_20180227_00000143_B.fits
03/20/2018  05:13 PM        16,663,680 M42_20180227_00000144_R.fits
03/20/2018  05:13 PM        16,663,680 M42_20180227_00000145_G.fits
03/20/2018  05:13 PM        16,663,680 M42_20180227_00000146_B.fits
03/20/2018  05:13 PM        16,663,680 M42_20180227_00000147_R.fits
03/20/2018  05:13 PM        16,663,680 M42_20180227_00000148_R.fits
03/20/2018  05:13 PM        16,663,680 M42_20180227_00000149_G.fits
03/20/2018  05:13 PM        16,663,680 M42_20180227_00000150_G.fits
03/20/2018  05:13 PM        16,663

In [63]:
biasfiles = glob('M42_Feb2017_Images/*Bias*')
nbias = len(biasfiles)
darkfiles = glob('M42_Feb2017_Images/*Dark*')
ndark = len(darkfiles)
flatred = glob('M42_Feb2017_Images/*R.fits*')
nrflat = len(flatred)
flatgreen = glob('M42_Feb2017_Images/*G.fits*')
ngflat = len(flatgreen)
flatblue = glob('M42_Feb2017_Images/*B.fits*')
nbflat = len(flatblue)
datafiles = glob('M42_Feb2017_Images/*M42*')
ndata = len(datafiles)

In [64]:
nx,ny = fits.open(datafiles[0])[0].data.shape
print(nx,ny)

2504 3326


In [1]:
#observation = {'dark':{'frames':np.zeros([nx,ny,ndark]),'files':darkfiles,'hdrs':[],
#                       'med':np.zeros(ndark),'std':np.zeros(ndark)},
#             'bias':{'frames':np.zeros([nx,ny,nbias]),'files':biasfiles,'hdrs':[],
#                     'med':np.zeros(nbias),'std':np.zeros(nbias)},
#              'rflat':{'frames':np.zeros([nx,ny,nrflat]),'files':flatred,'hdrs':[],
#                     'med':np.zeros(nrflat),'std':np.zeros(nrflat)},
#              'gflat':{'frames':np.zeros([nx,ny,ngflat]),'files':flatgreen,'hdrs':[],
#                     'med':np.zeros(ngflat),'std':np.zeros(ngflat)},
#              'bflat':{'frames':np.zeros([nx,ny,nbflat]),'files':flatblue,'hdrs':[],
#                     'med':np.zeros(nbflat),'std':np.zeros(nbflat)},
#              'data':{'frames':np.zeros([nx,ny,ndata]),'files':datafiles,'hdrs':[],
#                     'med':np.zeros(ndata),'std':np.zeros(ndata)}}

In [ ]:
#for imagetype in observation.keys():
#    print(imagetype,observation[imagetype]['frames'].shape)
#    for i,file in enumerate(observation[imagetype]['files']):
#        f = fits.open(file)
        #dat = fits.open(file)[0].data.copy()
        #observation[imagetype]['frames'][:,:,i] = dat
        #observation[imagetype]['med'][i] = np.median(dat)
        #observation[imagetype]['std'][i] = np.std(dat)
#        f.close()

In [65]:
def ShowImage(image,med=None,std=None,sigma=3,colorbar=True,vmin=None,vmax=None):
    if med==None:
        med = np.median(image)
    if std==None:
        std = np.std(image)
    plt.figure(figsize=(10,10))
    if vmin == None:
        vmin=med-sigma*std
    if vmax == None:
        vmax=med+sigma*std
    plt.imshow(image,vmin=vmin,vmax=vmin)
    if colorbar:
        plt.colorbar()
    plt.show()
def ShowObsFrame(observation,imagetype,index,sigma=3):
    dat = observation[imagetype]['frames'][:,:,index]
    med = observation[imagetype]['med'][index]
    std = observation[imagetype]['std'][index]
    ShowImage(dat,med=med,std=std,sigma=sigma)
    return

In [66]:
flatred

['M42_Feb2017_Images\\M42_20180227_00000141_R.fits',
 'M42_Feb2017_Images\\M42_20180227_00000144_R.fits',
 'M42_Feb2017_Images\\M42_20180227_00000147_R.fits',
 'M42_Feb2017_Images\\M42_20180227_00000148_R.fits',
 'M42_Feb2017_Images\\M42_20180227_00000162_R.fits',
 'M42_Feb2017_Images\\M42_20180227_00000163_R.fits']

In [68]:
#pulls in red images, makes red flat
for i, file in enumerate(flatred):
    with fits.open(file) as f:
        dat = f[0].data.copy()
        hdr = f[0].header.copy()
flatred = np.median(dat, axis = 0) 

In [34]:
#pulls in green images, makes green flat
for i, file in enumerate(flatgreen):
    with fits.open(file) as f:
        dat = f[0].data.copy()
        hdr = f[0].header.copy()
flatgreen = np.median(dat, axis = 0)

In [69]:
#pulls in blue images, makes blue flat
for i, file in enumerate(flatblue):
    with fits.open(file) as f:
        dat = f[0].data.copy()
        hdr = f[0].header.copy()
flatblue = np.median(dat, axis = 0)

In [71]:
#make mean of dark
for i, file in enumerate(darkfiles):
    with fits.open(file) as f:
        dat = f[0].data.copy()
        hdr = f[0].header.copy()
darkfiles = np.mean(dat, axis = 0)

In [72]:
for i, file in enumerate(biasfiles):
    with fits.open(file) as f:
        dat = f[0].data.copy()
        hdr = f[0].header.copy()
biasfiles = np.mean(dat, axis = 0)

In [46]:
darkfiles = darkfiles - biasfiles

flatred = flatred - darkfiles
flatgreen = flatgreen - darkfiles
flatblue = flatblue - darkfiles

normflatr = (flatred - biasfiles)/(np.median(flatred-biasfiles)) #red
normflatg = (flatgreen - biasfiles)/(np.median(flatgreen-biasfiles)) #green
normflatb = (flatblue - biasfiles)/(np.median(flatgreen-biasfiles))#blue

In [58]:
image_processed_r = (flatred[0]-biasfiles)/normflatr
image_processed_g = (flatgreen[0]-biasfiles)/normflatg
image_processed_b = (flatblue[0]-biasfiles)/normflatb

In [52]:
normflats = fits.PrimaryHDU(normflatr)
normflats.writeto('flats/flat-r.fits',overwrite=True)

normflats = fits.PrimaryHDU(normflatg)
normflats.writeto('flats/flat-g.fits',overwrite=True)

normflats = fits.PrimaryHDU(normflatb)
normflats.writeto('flats/flat-b.fits', overwrite=True)

In [59]:
processed_r = fits.PrimaryHDU(image_processed_r)
processed_r.writeto('flats/image_processed_r.fits',overwrite=True)

processed_g = fits.PrimaryHDU(image_processed_g)
processed_g.writeto('flats/image_processed_g.fits',overwrite=True)

processed_b = fits.PrimaryHDU(image_processed_b)
processed_b.writeto('flats/image_processed_b.fits', overwrite=True)

### The processed images that I have exported are attached. I am not sure where I went wrong but these files come up as a single line rather than a 2D image. Anyways I tried